In [1]:
import duckdb

# connect to  DuckDB database file
con = duckdb.connect("all_posts.duckdb")

# export first post date and body per user directly to a CSV
con.execute("""
    COPY (
        SELECT
            a.author_did,
            a.created_at AS first_post_created_at,
            a.body AS first_post_body
        FROM all_posts a
        INNER JOIN (
            SELECT
                author_did,
                MIN(created_at) AS first_post_created_at
            FROM all_posts
            GROUP BY author_did
        ) b
        ON a.author_did = b.author_did AND a.created_at = b.first_post_created_at
    ) TO 'first_posts_per_user.csv' (HEADER, DELIMITER ',');
""")

con.close()


In [2]:
import pandas as pd

# load csvs
users_df = pd.read_csv("sample_users.csv")
first_posts_df = pd.read_csv("first_posts_per_user.csv")

# merge on did
merged_df = users_df.merge(
    first_posts_df,
    how="inner",
    left_on="did",
    right_on="author_did"
)

#  cleanup
merged_df = merged_df.drop(columns=["author_did"])

# Save final result
merged_df.to_csv("sample_users_with_first_posts.csv", index=False)
print("CSV written: sample_users_with_first_posts.csv")


CSV written: sample_users_with_first_posts.csv


In [3]:
import pandas as pd

df = pd.read_csv("sample_users_with_first_posts.csv")

# Preview the first few rows
print(df.head())


                                did                      handle  \
0  did:plc:ooehugjick2vkzwlr64lephm        haleaziz.bsky.social   
1  did:plc:qy3kg6jvtlhenv5ojqurynwh     joshuacroke.bsky.social   
2  did:plc:ibqwicj6ersmvw36yldikozx  inspiringtimmy.bsky.social   
3  did:plc:f4ultoamz3y2m4drcjvhoecu  wilmonstouches.bsky.social   
4  did:plc:yzhdirwmonvaezndwnssoosd      kerrieneet.bsky.social   

                                                 bio               created_at  \
0  I cover DHS and immigration policy for the New...  2024-11-18 15:39:52.581   
1  Founder of 🏳️‍🌈 queerforcities.com / 🌈loveyour...  2024-11-15 11:45:12.223   
2  Best known for watching the Queen's Gambit fro...  2024-11-06 08:11:15.881   
3  same name but not rosh‘s ex | #OMAR : ”oh, tha...  2024-11-15 17:01:25.026   
4  Wild places in the SW and beyond. Where to go?...  2024-11-20 01:32:25.972   

        first_post_created_at  \
0  2024-11-16 20:04:59.812-05   
1  2024-11-15 07:36:53.045-05   
2  2024-11-

In [4]:
len(df)

23465

Total users with a first post: 23465
